In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "5596f562-286a-4e39-aa35-723f210a84f3",
   "metadata": {},
   "outputs": [],
   "source": [
    "import torch\n",
    "import torch.nn as nn\n",
    "import torch.nn.functional as F\n",
    "from torch.utils.data import TensorDataset\n",
    "import torch.optim as optim\n",
    "import torchvision\n",
    "from torchvision import datasets\n",
    "import torchvision.transforms as transforms\n",
    "import matplotlib.pyplot as plt\n",
    "from tqdm import tqdm"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "5f8981b6-5194-4dce-891a-febe26d26ac3",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "cuda:0\n"
     ]
    }
   ],
   "source": [
    "device = torch.device(\"cuda:0\" if torch.cuda.is_available() else \"cpu\")\n",
    "print(device)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "8462da36-52db-4fdb-8e0d-4b78aa5d2435",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'NVIDIA A100-SXM4-80GB MIG 1g.20gb'"
      ]
     },
     "execution_count": 3,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "torch.cuda.get_device_name(0)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "afe4c059-c3dc-4a57-a657-a72fdf573b66",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "False , cpu\n"
     ]
    }
   ],
   "source": [
    "X_train = torch.IntTensor([0, 30, 50, 75, 70])\n",
    "print(X_train.is_cuda, \",\", X_train.device)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "eda6746c-6c18-4350-888c-8651abad09af",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "True , cuda:0\n"
     ]
    }
   ],
   "source": [
    "X_train = X_train.cuda()\n",
    "# Alternative method: specify the device using the variable\n",
    "# X_train = X_train.to(device)\n",
    "# Confirm that the Tensor is on the GPU now\n",
    "print(X_train.is_cuda, \",\", X_train.device)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "id": "fa4778e3-4b9f-417f-bfc5-c317fc517066",
   "metadata": {},
   "outputs": [],
   "source": [
    "batch_size = 100\n",
    "\n",
    "class SimpleNet(nn.Module):\n",
    "    def __init__(self):\n",
    "        super(SimpleNet, self).__init__()\n",
    "        self.fc1 = nn.Linear(784, 784)\n",
    "        self.fc2 = nn.Linear(784, 10)\n",
    "\n",
    "    def forward(self, x):\n",
    "        x = x.view(batch_size, -1)\n",
    "        x = self.fc1(x)\n",
    "        x = F.relu(x)\n",
    "        x = self.fc2(x)\n",
    "        output = F.softmax(x, dim=1)\n",
    "        return output"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "d4ad8baa-9e75-4c69-aca3-dd2fd82e0c38",
   "metadata": {},
   "outputs": [],
   "source": [
    "model = SimpleNet().to(device)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "id": "0b9f3e4c-d4d7-4894-b118-7fa10e220a58",
   "metadata": {},
   "outputs": [],
   "source": [
    "train_transform=transforms.Compose([\n",
    "        transforms.ToTensor(),\n",
    "        transforms.Normalize((0.1307,), (0.3081,))\n",
    "        ])\n",
    "\n",
    "test_transform=transforms.Compose([\n",
    "        transforms.ToTensor(),\n",
    "        transforms.Normalize((0.1307,), (0.3081,)),\n",
    "        ])\n",
    "\n",
    "# Set up a training data set\n",
    "trainset = datasets.FashionMNIST('./data', train=True, download=True,\n",
    "                  transform=train_transform)\n",
    "train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,\n",
    "                                          shuffle=False, num_workers=2)\n",
    "\n",
    "# Set up a test data set\n",
    "testset = datasets.FashionMNIST('./data', train=False,\n",
    "                  transform=test_transform)\n",
    "test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,\n",
    "                                        shuffle=False, num_workers=2)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "id": "bcbdb9d0-2129-4ac5-a189-08489f19a63a",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...",
      "text/plain": [
       "<Figure size 800x800 with 9 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "labels_map = {\n",
    "    0: \"T-Shirt\",\n",
    "    1: \"Trouser\",\n",
    "    2: \"Pullover\",\n",
    "    3: \"Dress\",\n",
    "    4: \"Coat\",\n",
    "    5: \"Sandal\",\n",
    "    6: \"Shirt\",\n",
    "    7: \"Sneaker\",\n",
    "    8: \"Bag\",\n",
    "    9: \"Ankle Boot\",\n",
    "}\n",
    "\n",
    "# Plotting 9 random different items from the training data set, trainset.\n",
    "figure = plt.figure(figsize=(8, 8))\n",
    "for i in range(1, 3 * 3 + 1):\n",
    "    sample_idx = torch.randint(len(trainset), size=(1,)).item()\n",
    "    img, label = trainset[sample_idx]\n",
    "    figure.add_subplot(3, 3, i)\n",
    "    plt.title(labels_map[label])\n",
    "    plt.axis(\"off\")\n",
    "    plt.imshow(img.view(28,28), cmap=\"gray\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "id": "9ef4f746-f342-4adf-bce6-cfdef0c8a1df",
   "metadata": {},
   "outputs": [],
   "source": [
    "def train(model, device, train_loader, optimizer, epoch):\n",
    "    \"\"\"Model training function\"\"\"\n",
    "    model.train()\n",
    "    print(device)\n",
    "    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):\n",
    "        data, target = data.to(device), target.to(device)\n",
    "        optimizer.zero_grad()\n",
    "        output = model(data)\n",
    "        loss = F.nll_loss(output, target)\n",
    "        loss.backward()\n",
    "        optimizer.step()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "id": "fd843896-03b1-4cb7-a5c0-b420bba2cff9",
   "metadata": {},
   "outputs": [],
   "source": [
    "def test(model, device, test_loader):\n",
    "    \"\"\"Model evaluating function\"\"\"\n",
    "    model.eval()\n",
    "    test_loss = 0\n",
    "    correct = 0\n",
    "    with torch.no_grad():\n",
    "        for data, target in test_loader:\n",
    "            data, target = data.to(device), target.to(device)\n",
    "            output = model(data)\n",
    "            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss\n",
    "            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability\n",
    "            correct += pred.eq(target.view_as(pred)).sum().item()\n",
    "\n",
    "    test_loss /= len(test_loader.dataset)\n",
    "\n",
    "    print('\\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\\n'.format(\n",
    "        test_loss, correct, len(test_loader.dataset),\n",
    "        100. * correct / len(test_loader.dataset)))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "id": "f47b42fe-4905-48fe-ad88-7f903e281ea1",
   "metadata": {},
   "outputs": [],
   "source": [
    "def test(model, device, test_loader):\n",
    "    model.eval()\n",
    "    test_loss = 0\n",
    "    correct = 0\n",
    "    # Use the no_grad method to increase computation speed\n",
    "    # since computing the gradient is not necessary in this step.\n",
    "    with torch.no_grad():\n",
    "        for data, target in test_loader:\n",
    "            data, target = data.to(device), target.to(device)\n",
    "            output = model(data)\n",
    "            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss\n",
    "            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability\n",
    "            correct += pred.eq(target.view_as(pred)).sum().item()\n",
    "\n",
    "    test_loss /= len(test_loader.dataset)\n",
    "\n",
    "    print('\\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\\n'.format(\n",
    "        test_loss, correct, len(test_loader.dataset),\n",
    "        100. * correct / len(test_loader.dataset)))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "id": "a4d951f1-ade1-44be-93f1-de53051109eb",
   "metadata": {},
   "outputs": [],
   "source": [
    "EPOCHS = 5\n",
    "# our optimization strategy used in training.\n",
    "optimizer = optim.Adadelta(model.parameters(), lr=0.01)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "id": "c6072415-652e-4fb7-a0e4-57467ae23d23",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "EPOCH: 1\n",
      "cuda:0\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "600it [00:06, 89.07it/s]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "Test set: Average loss: -0.5645, Accuracy: 6205/10000 (62%)\n",
      "\n",
      "EPOCH: 2\n",
      "cuda:0\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "600it [00:06, 91.25it/s]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "Test set: Average loss: -0.6295, Accuracy: 6881/10000 (69%)\n",
      "\n",
      "EPOCH: 3\n",
      "cuda:0\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "600it [00:06, 94.15it/s]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "Test set: Average loss: -0.6669, Accuracy: 7086/10000 (71%)\n",
      "\n",
      "EPOCH: 4\n",
      "cuda:0\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "600it [00:06, 91.84it/s]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "Test set: Average loss: -0.6833, Accuracy: 7143/10000 (71%)\n",
      "\n",
      "EPOCH: 5\n",
      "cuda:0\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "600it [00:06, 90.96it/s] \n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "Test set: Average loss: -0.6974, Accuracy: 7196/10000 (72%)\n",
      "\n"
     ]
    }
   ],
   "source": [
    "for epoch in range(1, EPOCHS + 1):\n",
    "        print( f\"EPOCH: {epoch}\")\n",
    "        train(model, device, train_loader, optimizer, epoch)\n",
    "        test(model, device, test_loader)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "id": "b3fb979c-ce84-4657-996e-8509912581de",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Saving the model's weights!\n",
    "torch.save(model.state_dict(), \"mnist_fashion_SimpleNet.pt\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "4cbda6a9-b468-45f6-a346-3f86fa2e9214",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.8"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}